# Ferramentas para NLP

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cin').getOrCreate()

In [2]:
from pyspark.sql.functions import regexp_replace, lower, monotonically_increasing_id, lit

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
df = spark.read.option("encoding", "ISO-8859-1").text(['data.txt', 'class.txt', 'newtest.txt'])

In [5]:
df = df.withColumn("value", regexp_replace(lower(df["value"]), "[$&+,:;=?@#|'<>.-^*()%!]", ""))

In [6]:
df = df.select('*').withColumn("id", monotonically_increasing_id())

In [7]:
df.show()

+--------------------+-----------+
|               value|         id|
+--------------------+-----------+
|classificacao reg...|          0|
|aqui um arquivo d...| 8589934592|
|    novão pra checar|17179869184|
+--------------------+-----------+



## Tokenizer

Separação em tokens.

In [8]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [9]:
tokenizer = Tokenizer(inputCol="value", outputCol="words")

#extrai o token de acordo com o que foi especificado - aqui foi especificado
#regexTokenizer = RegexTokenizer(inputCol="value", outputCol="words", pattern="\\W")
# pattern="\\w+", gaps(False)

countTokens = udf(lambda words: len(words), IntegerType())

tokenized = tokenizer.transform(df)
tokenized.toPandas()


,value,id,words
0,classificacao regressao modelos sao legais,0,"[classificacao, regressao, modelos, sao, legais]"
1,aqui um arquivo de texto,8589934592,"[aqui, um, arquivo, de, texto]"
2,novão pra checar,17179869184,"[novão, pra, checar]"


In [62]:
tokenized.select("value", "words")\
    .withColumn("tokens", countTokens(col("words"))).toPandas()


,value,words,tokens
0,classificacao regressao modelos sao legais,"[classificacao, regressao, modelos, sao, legais]",5
1,aqui um arquivo de texto,"[aqui, um, arquivo, de, texto]",5
2,novão pra checar,"[novão, pra, checar]",3



## Removendo Stop Words

Stop Words são palavras que devem ser excluídas da entrada, normalmente porque as palavras aparecem com frequência e não têm tanto significado.

StopWordsRemover toma como entrada uma sequência de strings (por exemplo, a saída de um Tokenizer) e elimina todas as palavras de parada das seqüências de entrada. A lista de Stop Words é especificada pelo parâmetro stopWords. As Stop Words padrões para alguns idiomas são acessíveis chamando StopWordsRemover.loadDefaultStopWords(idioma), para o qual as opções disponíveis são “danish”, “dutch”, “english”, “finnish”, “french”, “german”, “hungarian”, “italian”, “norwegian”, “portuguese”, “russian”, “spanish”, “swedish” and “turkish”. Um parâmetro booleano caseSensitive indica se as correspondências devem diferenciar maiúsculas de minúsculas (falso por padrão).

In [10]:
from pyspark.ml.feature import StopWordsRemover
language="portuguese"
remover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords=StopWordsRemover.loadDefaultStopWords(language))
remover = remover.transform(tokenized)

In [11]:
remover.toPandas()

,value,id,words,filtered
0,classificacao regressao modelos sao legais,0,"[classificacao, regressao, modelos, sao, legais]","[classificacao, regressao, modelos, sao, legais]"
1,aqui um arquivo de texto,8589934592,"[aqui, um, arquivo, de, texto]","[aqui, arquivo, texto]"
2,novão pra checar,17179869184,"[novão, pra, checar]","[novão, pra, checar]"


## N-Gramas

NGram toma como entrada uma seqüência strings (por exemplo, a saída de um Tokenizer). O parâmetro n é usado para determinar o número de termos em cada n-grama. A saída consistirá de uma sequência de n-gramas, onde cada n-grama é representado por uma string delimitada por espaço de n palavras consecutivas. Se a sequência de entrada contiver menos de n cadeias, nenhuma saída será produzida.


In [65]:
from pyspark.ml.feature import NGram

ngram = NGram(n=2, inputCol="filtered", outputCol="ngrams")
ngramDataFrame = ngram.transform(remover)

ngramDataFrame.select("ngrams").show(truncate=False)

+---------------------------------------------------------------------+
|ngrams                                                               |
+---------------------------------------------------------------------+
|[classificacao regressao, regressao modelos, modelos sao, sao legais]|
|[aqui arquivo, arquivo texto]                                        |
|[novão pra, pra checar]                                              |
+---------------------------------------------------------------------+



_______
# TF-IDF
_______

In [11]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

sentenceData = spark.createDataFrame([
    (0.0, "Ola ja ouvi falar de Spark"),
    (1.0, "Eu gostaria de usar datascience em tudo"),
    (2.0, "Classificacao,regressao,modelos,sao,legais")
], ["label", "sentence"])

sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Ola ja ouvi falar...|
|  1.0|Eu gostaria de us...|
|  2.0|Classificacao,reg...|
+-----+--------------------+



In [12]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)
wordsData.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Ola ja ouvi falar...|[ola, ja, ouvi, f...|
|  1.0|Eu gostaria de us...|[eu, gostaria, de...|
|  2.0|Classificacao,reg...|[classificacao,re...|
+-----+--------------------+--------------------+



## CountVectorizer

O CountVectorizer selecionará as melhores palavras com um tamanho igual ao valor especificado pelo parâmetro vocabSize, ordenadas pela frequência do termo em todo o corpus. Um parâmetro opcional minDF também afeta o processo de ajuste, especificando o número mínimo dos documentos em que um termo deve aparecer para ser incluído no vocabulário. 

In [13]:
from pyspark.ml.feature import CountVectorizer

In [14]:
cv = CountVectorizer(inputCol="filtered", outputCol="features")
#minDf o termo só aparece se estiver em mais documentos que o minimo.
model = cv.fit(remover)

result = model.transform(remover)


In [16]:
text = ['data.txt', 'class.txt', 'newtest.txt']

In [20]:
selected = result.select('features')

In [21]:
links = [list(map(int, selected.collect()[ind][0].toArray())) for ind, x in enumerate(text)]

In [26]:
dict_vector = {}

In [27]:
for ind, x in enumerate(text):
    dict_vector[text[ind]] = list(map(int, selected.collect()[ind][0].toArray()))

In [28]:
dict_vector

{'class.txt': [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
 'data.txt': [0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0],
 'newtest.txt': [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0]}

In [17]:
list(map(int, result.select('features').collect()[1][0].toArray()))

[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]

In [16]:
model.vocabulary

['arquivo',
 'checar',
 'legais',
 'sao',
 'pra',
 'regressao',
 'novão',
 'classificacao',
 'aqui',
 'modelos',
 'texto']